**1. INSTALLER LES LIBRAIRIES**

In [174]:
# Exécuter seulement si ce n'est pas encore intallé

# !pip install langdetect
# !pip install textblob
# !pip install -U textblob-fr

**2. IMPORTER LES LIBRAIRIES**

In [175]:
import pandas as pd
import numpy as np

import datetime
import re

from langdetect import detect

from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

import warnings
warnings.filterwarnings('ignore')

**3. IMPORTER LE FICHIER**

In [176]:
# Import du fichier .csv obtenu suite au Webscrapping avec les noms de colonnes associées.
col_names = ['nom', 'notes', 'pays', 'nbr_avis', 'date_publication', 'date_experience', 'commentaire', 'titre',] 
df = pd.read_csv("1. Résultats Wepscrapping.csv", names = col_names)

df.info()
display(df.head())
print("Nombre de valeurs nulles :", df.isna().sum().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83660 entries, 0 to 83659
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   nom               83659 non-null  object
 1   notes             83660 non-null  int64 
 2   pays              83660 non-null  object
 3   nbr_avis          83660 non-null  object
 4   date_publication  83660 non-null  object
 5   date_experience   83660 non-null  object
 6   commentaire       83660 non-null  object
 7   titre             83659 non-null  object
dtypes: int64(1), object(7)
memory usage: 5.1+ MB


,nom,notes,pays,nbr_avis,date_publication,date_experience,commentaire,titre
0,Andy Sisley,2,GB,3 reviews,3 days ago,"Date of experience: March 29, 2024",Very poor customer service when a seller fails...,Very poor customer service when a…
1,GUY PEYRONNENC,5,FR,7 reviews,2 days ago,"Date of experience: April 09, 2024",commande simple et rapide,commandee simple et rapide
2,Client,5,FR,9 reviews,20 hours ago,"Date of experience: April 10, 2024",super merci,rapide et simple
3,bernard auge,5,FR,1 review,4 days ago,"Date of experience: April 07, 2024",tres bon service,rapide et simple
4,notzli walter,5,FR,12 reviews,5 days ago,"Date of experience: April 06, 2024",Cdiscount allows false advertising on their pl...,super merci


Nombre de valeurs nulles : 2


**4. PRE-PROCESSING & FEATURE ENGINEERING_1**

*1) Suppression des lignes avec une valeur nulle*

In [177]:
df = df.dropna(axis=0, how='any')
# Vérification
print("Nombre de valeurs nulles restant :", df.isna().sum().sum())

Nombre de valeurs nulles restant : 0


*2) Conserver uniquement les avis provenant de France*

In [178]:
print("Nombre d'avis provenant d'un pays autre que la France :", len(df.loc[(df['pays'] != 'FR')]))
df = df.drop(df[(df['pays'] != 'FR')].index)
# Réinitialiser l'index
df = df.reset_index(drop=True)
# Vérification
print("Nombre d'avis restant provenant d'un pays autre que la France :", len(df.loc[(df['pays'] != 'FR')]))

Nombre d'avis provenant d'un pays autre que la France : 1712
Nombre d'avis restant provenant d'un pays autre que la France : 0


*3) Fusionner les colonnes 'commentaire' et 'titre'*

In [179]:
# Nous avons déjà supprimer les lignes avec une valeur nulle.
# Ici, les lignes sans commentaires sont indiquées par les termes "No comment" dans la colonne "commentaire".
print("Nombre de lignes sans commentaires :", df.loc[(df['commentaire'] == 'No comment')].shape[0])

# Fusionner les colonnes 'commentaires' et 'titre' dans une nouvelle colonne 'commentaires'
commentaires = []
for i in df.index :
    # S'il n'y pas de commentaires, définir le titre comme le commentaire
    if (df['commentaire'][i] == "No comment") :
        commentaires.append(df['titre'][i])
    # Si le titre et le commentaire sont identiques, conserver uniquement le commentaire
    elif (df['titre'][i] == df['commentaire'][i]):
        commentaires.append(df['commentaire'][i])
    # Sinon, fusionner le titre et le commentaire
    else : 
        commentaires.append(df['titre'][i] + " " + df['commentaire'][i])

# Ajouter la nouvelle colonne 'commentaires' au Dataframe
nouvelle_colonne = pd.DataFrame(commentaires, columns=["commentaires"])
df = pd.concat([df, nouvelle_colonne], axis = 1)

# Vérification
df.head()

Nombre de lignes sans commentaires : 22499


,nom,notes,pays,nbr_avis,date_publication,date_experience,commentaire,titre,commentaires
0,GUY PEYRONNENC,5,FR,7 reviews,2 days ago,"Date of experience: April 09, 2024",commande simple et rapide,commandee simple et rapide,commandee simple et rapide commande simple et ...
1,Client,5,FR,9 reviews,20 hours ago,"Date of experience: April 10, 2024",super merci,rapide et simple,rapide et simple super merci
2,bernard auge,5,FR,1 review,4 days ago,"Date of experience: April 07, 2024",tres bon service,rapide et simple,rapide et simple tres bon service
3,notzli walter,5,FR,12 reviews,5 days ago,"Date of experience: April 06, 2024",Cdiscount allows false advertising on their pl...,super merci,super merci Cdiscount allows false advertising...
4,Cat Deshallus,5,FR,12 reviews,5 days ago,"Date of experience: April 06, 2024",Hello. can you plase advise if this is your ma...,simple et efficace,simple et efficace Hello. can you plase advise...


*4) Conserver uniquement les commentaires en français*

In [180]:
# Certains commentaires provenant de France sont dans une autre langue, nous les supprimons :
for i in df.index :
  try :
      if detect(df["commentaires"][i]) != "fr" :
          df = df.drop(index = i)
  except : 
      df = df.drop(index = i)

# Réinitialiser l'index
df = df.reset_index(drop=True)

*5) Transformer la colonne 'nbr_avis' et la mettre au bon format*

In [181]:
# '3 reviews' (object) >>> 3 (int64)
df['nbr_avis'] = df['nbr_avis'].apply(lambda x : x.split(' ')[0])
df['nbr_avis'] = df['nbr_avis'].astype('int64')
# Vérification
print("Format de la colonne 'nbr_avis' :", df['nbr_avis'].dtype)

Format de la colonne 'nbr_avis' : int64


*6) Mettre la colonnes 'date_experience' au format datetime*

In [182]:
df['date_experience'] = df['date_experience'].apply(lambda date: date.split(': ')[1])
df['date_experience'] = pd.to_datetime(df['date_experience'])
# Vérification
print("Format de la colonne 'date_experience' :", df['date_experience'].dtype)

Format de la colonne 'date_experience' : datetime64[ns]


*7) Création de la variable 'ponctuation'*

In [183]:
# La variable "ponctuation" comptabilise le nombre de '!', '?' et '...' dans un commentaire :
def compteur_ponctuation(str):
    r = re.compile("[!\?]") 
    r1 = re.compile("[...]{3,}") 
    str1 = r.findall(str)
    str2 = r1.findall(str)
    return len(str1)+len(str2)
    
# Ajout de la colonne dans le DataFrame
df['ponctuation'] = df['commentaires'].apply(lambda x : compteur_ponctuation(x))

*8) Création de la variable 'sentiment'*

In [184]:
# Variable quantitative
def sentiment(commentaire):
    blob = tb(commentaire)
    return blob.sentiment[0]

# Ajout de la colonne dans le DataFrame
df['sentiment'] = df['commentaires'].apply(lambda x : sentiment(x))

*9) Création de la variable 'sentiment_labels'*

In [185]:
# Variable qualitative : Classer les sentiments selon les labels 'Positif', 'Negatif' ou 'Neutre'
senti_list = []
for i in range(len(df['sentiment'])):
    if (df['sentiment'][i] > 0):
        senti_list.append('Positif')
    elif (df['sentiment'][i] < 0):
        senti_list.append('Negatif')
    else:
        senti_list.append('Neutre')

# Ajout de la colonne dans le DataFrame
df["sentiment_labels"] = senti_list

*10) Vérification finale*

In [186]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73846 entries, 0 to 73845
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   nom               73846 non-null  object        
 1   notes             73846 non-null  int64         
 2   pays              73846 non-null  object        
 3   nbr_avis          73846 non-null  int64         
 4   date_publication  73846 non-null  object        
 5   date_experience   73846 non-null  datetime64[ns]
 6   commentaire       73846 non-null  object        
 7   titre             73846 non-null  object        
 8   commentaires      73846 non-null  object        
 9   ponctuation       73846 non-null  int64         
 10  sentiment         73846 non-null  float64       
 11  sentiment_labels  73846 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(7)
memory usage: 6.8+ MB


,nom,notes,pays,nbr_avis,date_publication,date_experience,commentaire,titre,commentaires,ponctuation,sentiment,sentiment_labels
0,GUY PEYRONNENC,5,FR,7,2 days ago,2024-04-09,commande simple et rapide,commandee simple et rapide,commandee simple et rapide commande simple et ...,0,0.225000,Positif
1,bernard auge,5,FR,1,4 days ago,2024-04-07,tres bon service,rapide et simple,rapide et simple tres bon service,0,0.483333,Positif
2,alain bruno,5,FR,3,"Mar 26, 2024",2024-03-20,No comment,Honorable bon suivi !,Honorable bon suivi !,1,0.687500,Positif
3,Martin Maurice,1,FR,41,2 days ago,2024-04-09,Cdiscount cautionne l'escroquerie en ce sens o...,Cdiscount cautionne l'escroquerie en ce…,Cdiscount cautionne l'escroquerie en ce… Cdisc...,1,0.450000,Positif
4,PHILIPPE PLENET,5,FR,13,7 hours ago,2024-04-11,"Simple, rapide, prix raisonnables et €commerce...",Simple,"Simple Simple, rapide, prix raisonnables et €c...",0,0.190000,Positif


**5. DATAFRAME DANS UN FICHIER .CSV**

In [187]:
df.to_csv("2. Pre-processing & Feature engineering_1.csv", index=False)